In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
line_coords = []
with open('data/input_5.txt') as f:
    line = f.readline() 
    while  line:
        start, end = line.strip('\n').split(' -> ')
        start = [int(i) for i in start.split(',')]
        end = [int(i) for i in end.split(',')]
        line_coords.append((start, end))
        
        line = f.readline() 
        

In [34]:
def create_flatten_idx(coord, num_cols):
    return num_cols*coord[1] + coord[0]

def count_multi_hit(pt_dict):
    tot=0
    for val in pt_dict.values():
        if val > 1:
            tot+=1
    return tot
    
    
def count_seen_pts(line_list):
    pt_array = {}
    max_corner = [0,0]
    for line in line_list:
        pt1, pt2 = line
        curr_max = [ max(pt1[i], pt2[i]) for i in range (2)]
        max_corner = [ max(max_corner[i], curr_max[i]) for i in range (2)]
    num_cols= max_corner[1]+1
    # just the horizontal and vertical lines
    for line in line_list:
        pt1, pt2 = line
        if (pt1[0] != pt2[0]) and (pt1[1] != pt2[1]):
            continue
        #index of change
        idx = 1 if (pt1[0] == pt2[0]) else 0
        
        start=min(pt1[idx],pt2[idx])
        end=max(pt1[idx],pt2[idx])
        
        for i in range(start,end+1):
            tmp= [0,0]
            tmp[idx] = i 
            tmp[(idx+1)%2] = pt1[(idx+1)%2]
            cur_pt = create_flatten_idx(tmp, num_cols)
            try:
                pt_array[cur_pt] +=1
            except:
                # if index issue create it
                pt_array[cur_pt] = 1
    tot_twice_h_v=count_multi_hit(pt_array)
    
#    return tot_twice_h_v

    #Add diagonals
               
    for line in line_list:
        pt1, pt2 = line
    
        diff = [pt2[i] -pt1[i] for i in range(2)]
        if np.abs(diff[0]) != np.abs(diff[1]):
            continue
            
        sign = [ -1 if x < 0 else 1 for x in diff]
        
        for i in range(abs(diff[0])+1):
            x =pt1[0]+sign[0]*i
            y =pt1[1]+sign[1]*i
            cur_pt = create_flatten_idx((x,y), num_cols)
            try:
                pt_array[cur_pt] +=1
            except:
                # if index issue create it
                pt_array[cur_pt] = 1
    
    tot_twice_h_v_d=count_multi_hit(pt_array)
    
    return tot_twice_h_v, tot_twice_h_v_d
    
    
    

In [35]:
count_seen_pts(line_coords)

(7414, 19676)

# Fist Way
- The use of numpy array is heavier storage
- going to a more sparse memory storage above and keep similar algorithm (assuming the try/except makes things easier than looking if index had been visited

In [10]:
max_corner = [0,0]
for line in line_coords:
    pt1, pt2 = line
    curr_max = [ max(pt1[i], pt2[i]) for i in range (2)]
    max_corner = [ max(max_corner[i], curr_max[i]) for i in range (2)]


In [11]:
max_corner

[990, 990]

In [17]:
seen_points = np.zeros([ i+1 for i in max_corner])
for line in line_coords:
    pt1, pt2 = line
    if (pt1[0] != pt2[0]) and (pt1[1] != pt2[1]):
        continue
    if (pt1[0] == pt2[0]):
        start=min(pt1[1],pt2[1])
        end=max(pt1[1],pt2[1])
        
        for i in range(start,end+1):
            seen_points[pt1[0]][i] += 1
    
    if (pt1[1] == pt2[1]):
        start=min(pt1[0],pt2[0])
        end=max(pt1[0],pt2[0])
        
        for i in range(start,end+1):
            seen_points[i][pt1[1]] += 1

In [21]:
(seen_points > 1).sum()

7414

In [29]:
## Add in the diagonal lines
for line in line_coords:
    pt1, pt2 = line
    if (pt1[0] == pt2[0]) or (pt1[1] == pt2[1]):
        continue
    
    diff = [pt2[i] -pt1[i] for i in range(2)]
    if np.abs(diff[0]) != np.abs(diff[1]):
        continue
    sign = [ -1 if x < 0 else 1 for x in diff]
    for i in range(abs(diff[0])+1):
        x =pt1[0]+sign[0]*i
        y =pt1[1]+sign[1]*i
        seen_points[x][y] += 1


In [30]:
(seen_points > 1).sum()

19676